In [3]:
import pandas as pd
import pickle
import statistics
from statistics import mean
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [7]:
with open('df_train.pkl', 'rb') as f:
    df_train = pickle.load(f)
df_train.shape

(69445, 2423)

In [8]:
with open('df_validation.pkl', 'rb') as f:
    df_validation = pickle.load(f)
df_validation.shape

(22537, 2423)

In [9]:
with open('df_test.pkl', 'rb') as f:
    df_test = pickle.load(f)
df_test.shape

(22289, 2423)

In [10]:
df_train_4 = df_train[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()

df_validation_4 = df_validation[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()

df_test_4 = df_test[[
'INTERVAL_IN_FILE',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_13',
'BIT_FLAG',
'MAX_VALUE',
'NTC',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'VALUE'

]].reset_index()



In [7]:
df_for_model_tab = pd.DataFrame(columns = ['validation metric'])
df_for_model_tab['validation metric'] = ('F1', 'Accuracy', 'Precision', 'Recall', 'max difference from error mean', 'mean difference from error mean', 'number of bigger than 0.2 differnece', 'number of bigger than 0.1 differnece')
df_for_model_tab

,validation metric
0,F1
1,Accuracy
2,Precision
3,Recall
4,max difference from error mean
5,mean difference from error mean
6,number of bigger than 0.2 differnece
7,number of bigger than 0.1 differnece


In [18]:
def error_per_session(train_data, test_data, model, model_name):

    X_train = train_data.loc[:, train_data.columns != 'VALUE']
    X_test = test_data.loc[:, test_data.columns != 'VALUE']
    number = preprocessing.LabelEncoder()
    X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)       
    X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)       

    y_train = train_data['VALUE']
    y_test = test_data['VALUE']

    
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    prediction_row_check = pd.DataFrame(columns = ['y pred', 'y'])
    prediction_row_check['y']=y_test
    prediction_row_check['y pred']=y_pred
    prediction_row_check['SESSION_INFO_ID']=X_test['SESSION_INFO_ID']
    prediction_row_check['is prediction wrong'] = (prediction_row_check['y pred'] - prediction_row_check['y'])**2
    
    prediction_session_check=pd.DataFrame(prediction_row_check.groupby('SESSION_INFO_ID')['is prediction wrong'].mean())
    prediction_session_check = prediction_session_check.reset_index()
    
    prediction_session_check['abs difference from average'] = abs(prediction_session_check['is prediction wrong']-statistics.mean(prediction_row_check['is prediction wrong']))
    
    f1 = metrics.f1_score(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    max_difference_from_error_mean = prediction_session_check['abs difference from average'].max()
    mean_difference_from_error_mean = prediction_session_check['abs difference from average'].mean()
    number_of_bigger_than_02_differnece = prediction_session_check[prediction_session_check['abs difference from average']>0.2]['SESSION_INFO_ID'].count()
    number_of_bigger_than_01_differnece = prediction_session_check[prediction_session_check['abs difference from average']>0.1]['SESSION_INFO_ID'].count()
    df_for_model_tab[model_name] = f1, accuracy, precision, recall, max_difference_from_error_mean , mean_difference_from_error_mean, number_of_bigger_than_02_differnece, number_of_bigger_than_01_differnece



In [19]:
error_per_session(df_train_4, df_validation_4, RandomForestClassifier(), 'Random Forest')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest
0,F1,0.977957
1,Accuracy,0.981852
2,Precision,0.980759
3,Recall,0.975172
4,max difference from error mean,0.175882
5,mean difference from error mean,0.022414
6,number of bigger than 0.2 differnece,0.000000
7,number of bigger than 0.1 differnece,1.000000


In [20]:
error_per_session(df_train_4, df_validation_4, GradientBoostingClassifier(), 'Gradient Boost')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost
0,F1,0.977957,0.967270
1,Accuracy,0.981852,0.973200
2,Precision,0.980759,0.975410
3,Recall,0.975172,0.959265
4,max difference from error mean,0.175882,0.184287
5,mean difference from error mean,0.022414,0.031394
6,number of bigger than 0.2 differnece,0.000000,0.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000


In [21]:
error_per_session(df_train_4, df_validation_4, AdaBoostClassifier(), 'Ada Boost')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost
0,F1,0.977957,0.967270,0.946152
1,Accuracy,0.981852,0.973200,0.957004
2,Precision,0.980759,0.975410,0.979519
3,Recall,0.975172,0.959265,0.914983
4,max difference from error mean,0.175882,0.184287,0.255174
5,mean difference from error mean,0.022414,0.031394,0.046640
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000


In [22]:
from sklearn.naive_bayes import GaussianNB

In [23]:
error_per_session(df_train_4, df_validation_4, GaussianNB(), 'Gaussian Naive Bayes')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost,Gaussian Naive Bayes
0,F1,0.977957,0.967270,0.946152,0.817446
1,Accuracy,0.981852,0.973200,0.957004,0.864978
2,Precision,0.980759,0.975410,0.979519,0.925051
3,Recall,0.975172,0.959265,0.914983,0.732266
4,max difference from error mean,0.175882,0.184287,0.255174,0.644843
5,mean difference from error mean,0.022414,0.031394,0.046640,0.123599
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000,2.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000,43.000000


In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
error_per_session(df_train_4, df_validation_4, KNeighborsClassifier(), 'KNN')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost,Gaussian Naive Bayes,KNN
0,F1,0.977957,0.967270,0.946152,0.817446,0.855209
1,Accuracy,0.981852,0.973200,0.957004,0.864978,0.894662
2,Precision,0.980759,0.975410,0.979519,0.925051,0.988579
3,Recall,0.975172,0.959265,0.914983,0.732266,0.753547
4,max difference from error mean,0.175882,0.184287,0.255174,0.644843,0.655259
5,mean difference from error mean,0.022414,0.031394,0.046640,0.123599,0.117612
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000,2.000000,2.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000,43.000000,45.000000


In [26]:
from sklearn.neural_network import MLPClassifier

In [27]:
error_per_session(df_train_4, df_validation_4, MLPClassifier(), 'MLP')
df_for_model_tab

C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost,Gaussian Naive Bayes,KNN,MLP
0,F1,0.977957,0.967270,0.946152,0.817446,0.855209,0.958299
1,Accuracy,0.981852,0.973200,0.957004,0.864978,0.894662,0.966056
2,Precision,0.980759,0.975410,0.979519,0.925051,0.988579,0.972238
3,Recall,0.975172,0.959265,0.914983,0.732266,0.753547,0.944755
4,max difference from error mean,0.175882,0.184287,0.255174,0.644843,0.655259,0.181408
5,mean difference from error mean,0.022414,0.031394,0.046640,0.123599,0.117612,0.038812
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000,2.000000,2.000000,0.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000,43.000000,45.000000,2.000000


In [28]:
df_test_at_least_2 = df_test[['INTERVAL_IN_FILE',
'SN',
'ACCL_X',
'ACCL_Y',
'BAND_ARR_1',
'BAND_ARR_2',
'BAND_ARR_3',
'BAND_ARR_4',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_9',
'BAND_ARR_10',
'BAND_ARR_11',
'BAND_ARR_12',
'BAND_ARR_13',
'BAND_ARR_14',
'BAND_ARR_15',
'BAND_ARR_16',
'BAND_ARR_17',
'BAND_ARR_21',
'BAND_ARR_25',
'BAND_ARR_27',
'BAND_ARR_28',
'BAND_ARR_30',
'BAND_ARR_32',
'BATTERY',
'BIT_FLAG',
'MAX_VALUE',
'MIC_TD_RAW',
'NTC',
'FIRST_PACKET_TS',
'NTC__mean',
'NTC__quantile__q_0.2',
'NTC__quantile__q_0.6',
'NTC__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_6__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_7__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_9__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_9__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_10__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_11__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_12__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_13__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_14__w_20__widths_(2, 5, 10, 20)',
'NTC__ar_coefficient__coeff_0__k_10',
'NTC__fft_coefficient__attr_"real"__coeff_1',
'NTC__fft_coefficient__attr_"real"__coeff_4',
'NTC__fft_coefficient__attr_"real"__coeff_52',
'NTC__fft_coefficient__attr_"real"__coeff_90',
'NTC__fft_coefficient__attr_"imag"__coeff_3',
'NTC__fft_coefficient__attr_"imag"__coeff_4',
'NTC__fft_coefficient__attr_"imag"__coeff_17',
'NTC__fft_coefficient__attr_"imag"__coeff_26',
'NTC__fft_coefficient__attr_"imag"__coeff_27',
'NTC__fft_coefficient__attr_"imag"__coeff_28',
'NTC__fft_coefficient__attr_"imag"__coeff_29',
'NTC__fft_coefficient__attr_"imag"__coeff_33',
'NTC__fft_coefficient__attr_"imag"__coeff_37',
'NTC__fft_coefficient__attr_"imag"__coeff_59',
'NTC__fft_coefficient__attr_"imag"__coeff_93',
'NTC__fft_coefficient__attr_"abs"__coeff_8',
'NTC__fft_coefficient__attr_"angle"__coeff_9',
'NTC__fft_coefficient__attr_"angle"__coeff_10',
'NTC__fft_coefficient__attr_"angle"__coeff_45',
'NTC__fft_coefficient__attr_"angle"__coeff_46',
'NTC__fft_coefficient__attr_"angle"__coeff_59',
'NTC__fft_coefficient__attr_"angle"__coeff_77',
'NTC__fft_coefficient__attr_"angle"__coeff_86',
'NTC__fft_coefficient__attr_"angle"__coeff_95',
'NTC__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',
'NTC__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
'BAND_ARR_2__sum_values',
'BAND_ARR_2__count_below_mean',
'BAND_ARR_2__spkt_welch_density__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"real"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"real"__coeff_3',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_3',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_5',
'BAND_ARR_2__fft_coefficient__attr_"abs"__coeff_1',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_4',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_9',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_12',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_40',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_49',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_51',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_64',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_65',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_70',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_81',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_89',
'BAND_ARR_2__range_count__max_1__min_-1',
'BAND_ARR_2__agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',
'BAND_ARR_2__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',
'BAND_ARR_8__sum_values',
'BAND_ARR_8__longest_strike_below_mean',
'BAND_ARR_8__c3__lag_1',
'BAND_ARR_8__quantile__q_0.8',
'BAND_ARR_8__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_3',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'BAND_ARR_8__fft_coefficient__attr_"abs"__coeff_0',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_2',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_7',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_14',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_15',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_38',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_58',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_62',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_64',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_65',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_69',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_72',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_78',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_95',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_96',
'BAND_ARR_8__max_langevin_fixed_point__m_3__r_30',
'VALUE'

]].reset_index()

df_validation_at_least_2 = df_validation[['INTERVAL_IN_FILE',
'SN',
'ACCL_X',
'ACCL_Y',
'BAND_ARR_1',
'BAND_ARR_2',
'BAND_ARR_3',
'BAND_ARR_4',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_9',
'BAND_ARR_10',
'BAND_ARR_11',
'BAND_ARR_12',
'BAND_ARR_13',
'BAND_ARR_14',
'BAND_ARR_15',
'BAND_ARR_16',
'BAND_ARR_17',
'BAND_ARR_21',
'BAND_ARR_25',
'BAND_ARR_27',
'BAND_ARR_28',
'BAND_ARR_30',
'BAND_ARR_32',
'BATTERY',
'BIT_FLAG',
'MAX_VALUE',
'MIC_TD_RAW',
'NTC',
'FIRST_PACKET_TS',
'NTC__mean',
'NTC__quantile__q_0.2',
'NTC__quantile__q_0.6',
'NTC__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_6__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_7__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_9__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_9__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_10__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_11__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_12__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_13__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_14__w_20__widths_(2, 5, 10, 20)',
'NTC__ar_coefficient__coeff_0__k_10',
'NTC__fft_coefficient__attr_"real"__coeff_1',
'NTC__fft_coefficient__attr_"real"__coeff_4',
'NTC__fft_coefficient__attr_"real"__coeff_52',
'NTC__fft_coefficient__attr_"real"__coeff_90',
'NTC__fft_coefficient__attr_"imag"__coeff_3',
'NTC__fft_coefficient__attr_"imag"__coeff_4',
'NTC__fft_coefficient__attr_"imag"__coeff_17',
'NTC__fft_coefficient__attr_"imag"__coeff_26',
'NTC__fft_coefficient__attr_"imag"__coeff_27',
'NTC__fft_coefficient__attr_"imag"__coeff_28',
'NTC__fft_coefficient__attr_"imag"__coeff_29',
'NTC__fft_coefficient__attr_"imag"__coeff_33',
'NTC__fft_coefficient__attr_"imag"__coeff_37',
'NTC__fft_coefficient__attr_"imag"__coeff_59',
'NTC__fft_coefficient__attr_"imag"__coeff_93',
'NTC__fft_coefficient__attr_"abs"__coeff_8',
'NTC__fft_coefficient__attr_"angle"__coeff_9',
'NTC__fft_coefficient__attr_"angle"__coeff_10',
'NTC__fft_coefficient__attr_"angle"__coeff_45',
'NTC__fft_coefficient__attr_"angle"__coeff_46',
'NTC__fft_coefficient__attr_"angle"__coeff_59',
'NTC__fft_coefficient__attr_"angle"__coeff_77',
'NTC__fft_coefficient__attr_"angle"__coeff_86',
'NTC__fft_coefficient__attr_"angle"__coeff_95',
'NTC__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',
'NTC__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
'BAND_ARR_2__sum_values',
'BAND_ARR_2__count_below_mean',
'BAND_ARR_2__spkt_welch_density__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"real"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"real"__coeff_3',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_3',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_5',
'BAND_ARR_2__fft_coefficient__attr_"abs"__coeff_1',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_4',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_9',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_12',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_40',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_49',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_51',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_64',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_65',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_70',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_81',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_89',
'BAND_ARR_2__range_count__max_1__min_-1',
'BAND_ARR_2__agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',
'BAND_ARR_2__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',
'BAND_ARR_8__sum_values',
'BAND_ARR_8__longest_strike_below_mean',
'BAND_ARR_8__c3__lag_1',
'BAND_ARR_8__quantile__q_0.8',
'BAND_ARR_8__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_3',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'BAND_ARR_8__fft_coefficient__attr_"abs"__coeff_0',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_2',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_7',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_14',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_15',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_38',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_58',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_62',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_64',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_65',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_69',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_72',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_78',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_95',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_96',
'BAND_ARR_8__max_langevin_fixed_point__m_3__r_30',
'VALUE'

]].reset_index()

df_train_at_least_2 = df_train[['INTERVAL_IN_FILE',
'SN',
'ACCL_X',
'ACCL_Y',
'BAND_ARR_1',
'BAND_ARR_2',
'BAND_ARR_3',
'BAND_ARR_4',
'BAND_ARR_5',
'BAND_ARR_6',
'BAND_ARR_7',
'BAND_ARR_8',
'BAND_ARR_9',
'BAND_ARR_10',
'BAND_ARR_11',
'BAND_ARR_12',
'BAND_ARR_13',
'BAND_ARR_14',
'BAND_ARR_15',
'BAND_ARR_16',
'BAND_ARR_17',
'BAND_ARR_21',
'BAND_ARR_25',
'BAND_ARR_27',
'BAND_ARR_28',
'BAND_ARR_30',
'BAND_ARR_32',
'BATTERY',
'BIT_FLAG',
'MAX_VALUE',
'MIC_TD_RAW',
'NTC',
'FIRST_PACKET_TS',
'NTC__mean',
'NTC__quantile__q_0.2',
'NTC__quantile__q_0.6',
'NTC__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_6__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_7__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_7__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_8__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_9__w_5__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_9__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_10__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_11__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_12__w_10__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_12__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_13__w_20__widths_(2, 5, 10, 20)',
'NTC__cwt_coefficients__coeff_14__w_20__widths_(2, 5, 10, 20)',
'NTC__ar_coefficient__coeff_0__k_10',
'NTC__fft_coefficient__attr_"real"__coeff_1',
'NTC__fft_coefficient__attr_"real"__coeff_4',
'NTC__fft_coefficient__attr_"real"__coeff_52',
'NTC__fft_coefficient__attr_"real"__coeff_90',
'NTC__fft_coefficient__attr_"imag"__coeff_3',
'NTC__fft_coefficient__attr_"imag"__coeff_4',
'NTC__fft_coefficient__attr_"imag"__coeff_17',
'NTC__fft_coefficient__attr_"imag"__coeff_26',
'NTC__fft_coefficient__attr_"imag"__coeff_27',
'NTC__fft_coefficient__attr_"imag"__coeff_28',
'NTC__fft_coefficient__attr_"imag"__coeff_29',
'NTC__fft_coefficient__attr_"imag"__coeff_33',
'NTC__fft_coefficient__attr_"imag"__coeff_37',
'NTC__fft_coefficient__attr_"imag"__coeff_59',
'NTC__fft_coefficient__attr_"imag"__coeff_93',
'NTC__fft_coefficient__attr_"abs"__coeff_8',
'NTC__fft_coefficient__attr_"angle"__coeff_9',
'NTC__fft_coefficient__attr_"angle"__coeff_10',
'NTC__fft_coefficient__attr_"angle"__coeff_45',
'NTC__fft_coefficient__attr_"angle"__coeff_46',
'NTC__fft_coefficient__attr_"angle"__coeff_59',
'NTC__fft_coefficient__attr_"angle"__coeff_77',
'NTC__fft_coefficient__attr_"angle"__coeff_86',
'NTC__fft_coefficient__attr_"angle"__coeff_95',
'NTC__agg_linear_trend__attr_"intercept"__chunk_len_5__f_agg_"max"',
'NTC__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"',
'BAND_ARR_2__sum_values',
'BAND_ARR_2__count_below_mean',
'BAND_ARR_2__spkt_welch_density__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"real"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"real"__coeff_3',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_1',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_3',
'BAND_ARR_2__fft_coefficient__attr_"imag"__coeff_5',
'BAND_ARR_2__fft_coefficient__attr_"abs"__coeff_1',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_2',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_4',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_9',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_12',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_40',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_49',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_51',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_64',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_65',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_70',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_81',
'BAND_ARR_2__fft_coefficient__attr_"angle"__coeff_89',
'BAND_ARR_2__range_count__max_1__min_-1',
'BAND_ARR_2__agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"mean"',
'BAND_ARR_2__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"',
'BAND_ARR_8__sum_values',
'BAND_ARR_8__longest_strike_below_mean',
'BAND_ARR_8__c3__lag_1',
'BAND_ARR_8__quantile__q_0.8',
'BAND_ARR_8__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_3',
'BAND_ARR_8__fft_coefficient__attr_"real"__coeff_4',
'BAND_ARR_8__fft_coefficient__attr_"abs"__coeff_0',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_2',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_7',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_14',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_15',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_38',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_58',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_62',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_64',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_65',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_69',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_72',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_78',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_95',
'BAND_ARR_8__fft_coefficient__attr_"angle"__coeff_96',
'BAND_ARR_8__max_langevin_fixed_point__m_3__r_30',
'VALUE'

]].reset_index()


In [29]:
error_per_session(df_train_at_least_2, df_validation_at_least_2, MLPClassifier(), 'MLP - more features')
df_for_model_tab



C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\Users\Arie\AppData\Local\Temp/ipykernel_5564/2063655791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['SESSION_INFO_ID'] = number.fit_transform(X_test.SESSION_INFO_ID)


,validation metric,Random Forest,Gradient Boost,Ada Boost,Gaussian Naive Bayes,KNN,MLP,MLP - more features
0,F1,0.977957,0.967270,0.946152,0.817446,0.855209,0.958299,0.943103
1,Accuracy,0.981852,0.973200,0.957004,0.864978,0.894662,0.966056,0.953587
2,Precision,0.980759,0.975410,0.979519,0.925051,0.988579,0.972238,0.954736
3,Recall,0.975172,0.959265,0.914983,0.732266,0.753547,0.944755,0.931750
4,max difference from error mean,0.175882,0.184287,0.255174,0.644843,0.655259,0.181408,0.343768
5,mean difference from error mean,0.022414,0.031394,0.046640,0.123599,0.117612,0.038812,0.054034
6,number of bigger than 0.2 differnece,0.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000
7,number of bigger than 0.1 differnece,1.000000,2.000000,2.000000,43.000000,45.000000,2.000000,3.000000


In [1]:
grid_param = {
    'n_estimators': [20,  60, 100, 200, 300, 500],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_split': [1, 2, 3, 4, 5],
    'min_samples_leaf' : [1,2,3,4, 5],
    'max_features' : ['sqrt', 'log2', None],
}


In [4]:
from sklearn.model_selection import GridSearchCV

gd_sr = GridSearchCV(estimator=RandomForestClassifier(),
                     param_grid=grid_param,
                     scoring='f1',
                     cv=5,
                     n_jobs=-1)


In [11]:
X_train = df_train_4.loc[:, df_train_4.columns != 'VALUE']

number = preprocessing.LabelEncoder()
X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)       
y_train = df_train_4['VALUE']

gd_sr.fit(X_train, y_train)


C:\Users\Arie\AppData\Local\Temp/ipykernel_1644/2726015850.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['SESSION_INFO_ID'] = number.fit_transform(X_train.SESSION_INFO_ID)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [1, 2, 3, 4, 5],
                         'n_estimators': [20, 60, 100, 200, 300, 500]},
             scoring='f1')

In [12]:
best_parameters = gd_sr.best_params_
print(best_parameters)


{'criterion': 'gini', 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}


In [14]:
error_per_session(df_train_4, df_validation_4, RandomForestClassifier('criterion' = 'gini', 'max_features'= 'log2', min_samples_leaf= 1, min_samples_split= 2, n_estimators= 20), 'Random F Best Parameters')
df_for_model_tab

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (Temp/ipykernel_1644/1964352527.py, line 1)

In [ ]:
error_per_session(df_train_at_least_2, df_validation_at_least_2, MLPClassifier(), 'MLP - more features')
df_for_model_tab



In [ ]:
grid_param = {
    'loss': ['log_loss', 'deviance', 'exponential'],
    'learning_rate': [0.0, 0.1, 0.3, 0.5, 1.0, 2.0, 10.0, 100.0, 10000.0],
    'n_estimators': [1, 10, 100, 10000, 10000],
    'subsample' : [0.0, 0.25, 0.5, 0.75, 1.0],
    'criterion' : ['friedman_mse','squared_error', 'mse'],
}

gd_sr = GridSearchCV(estimator=GradientBoostingClassifier(),
                     param_grid=grid_param,
                     scoring='f1',
                     cv=5,
                     n_jobs=-1)

gd_sr.fit(X_train, y_train)
best_parameters = gd_sr.best_params_
print(best_parameters)


In [ ]:
grid_param = {
    'learning_rate': [0.0, 0.1, 0.3, 0.5, 1.0, 2.0, 10.0, 100.0, 10000.0],
    'n_estimators': [1, 10, 50, 100, 10000, 10000],
    'algorithm' : ['SAMME', 'SAMME.R']
}

gd_sr = GridSearchCV(estimator=AdaBoostClassifier(),
                     param_grid=grid_param,
                     scoring='f1',
                     cv=5,
                     n_jobs=-1)

gd_sr.fit(X_train, y_train)
best_parameters = gd_sr.best_params_
print(best_parameters)
